#Setting up

##Connecting to google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive

/content/drive


In [3]:
%cd MyDrive/StressPredictionProject/

/content/drive/MyDrive/StressPredictionProject


In [4]:
%pip install --user -U nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Importing libraries

In [5]:
import numpy as np
import sklearn 
import pandas as pd
import matplotlib as mp
import keras 
import csv 
import nltk
import pickle 

In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk import WordNetLemmatizer

In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

In [8]:
from keras.layers import SimpleRNN, Dense, Flatten, Conv1D, Dropout
from keras import regularizers

In [9]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#Extracting Data

##pickle

In [12]:
def save_data(data, file_name):
  with open(file_name, 'wb') as handle:
    pickle.dump(data, handle)

In [13]:
def get_data(file_name):
  with open(file_name, 'rb') as handle:
    data = pickle.load(handle)
  return data

##extracting raw data

In [ ]:
data = pd.read_csv("Stress.csv")
        

In [ ]:
data.head()

,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",1,0.8,1521614353
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",0,1.0,1527009817
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,1,0.8,1535935605
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",1,0.6,1516429555
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1,0.8,1539809005


Dropping unnecessary columns

In [ ]:
data = data.drop(["post_id", "social_timestamp", "sentence_range"], axis = 1)

##extracting labels

In [ ]:
labels = data["label"]

In [ ]:
labels = labels.to_numpy()

In [15]:
labels = get_data("labels.pkl")

In [ ]:
data = data.drop(["label", "confidence"], axis = 1)

In [ ]:
data.head()

,subreddit,text
0,ptsd,"He said he had not felt that way before, sugge..."
1,assistance,"Hey there r/assistance, Not sure if this is th..."
2,ptsd,My mom then hit me with the newspaper and it s...
3,relationships,"until i met my new boyfriend, he is amazing, h..."
4,survivorsofabuse,October is Domestic Violence Awareness Month a...


##tokenization

In [ ]:
def tokenize(data):
    new_data = []
    for element in data:
      new_data.append(word_tokenize(element))
    return new_data

In [ ]:
text = data["text"]

In [ ]:
new_text = tokenize(text)

In [ ]:
data["tokenized_text"] = new_text

##stopword removal

In [ ]:
stopwords = set(stopwords.words('english'))

In [ ]:
def remove_stopwords(data):
  new_data = []
  for words in data:
    filtered_sentence = [w for w in words if not w.lower() in stopwords]
    new_data.append(filtered_sentence)
  return new_data

In [ ]:
new_data = remove_stopwords(new_text)

##Stemming

In [ ]:
def stem(data):
  new_data = []
  stemmer = PorterStemmer()
  for words in data:
    new_words = [stemmer.stem(word) for word in words]
    new_data.append(new_words)
  return new_data


In [ ]:
new_data = stem(new_data)

In [ ]:
print(new_data)

[['said', 'felt', 'way', ',', 'sugget', 'go', 'rest', '..', 'trigger', 'ahead', 'youi', "'re", 'hypocondriac', 'like', ':', 'decid', 'look', '``', 'feel', 'doom', "''", 'hope', 'mayb', 'get', 'suck', 'rabbit', 'hole', 'ludicr', 'conspiraci', ',', 'stupid', '``', 'psychic', "''", 'test', 'new', 'age', 'b.s.', ',', 'someth', 'could', 'even', 'laugh', 'road', '.', ',', 'end', 'read', 'sens', 'doom', 'indic', 'variou', 'health', 'ailment', ';', 'one', 'prone', '..', 'top', '``', 'doom', "''", 'gloom', '..', "f'n", 'worri', 'heart', '.', 'happen', 'physic', '48', 'hour', '.'], ['hey', 'r/assist', ',', 'sure', 'right', 'place', 'post', '..', 'goe', '=', ')', "'m", 'current', 'student', 'intern', 'sandia', 'nation', 'lab', 'work', 'survey', 'help', 'improv', 'market', 'outreach', 'effort', 'mani', 'school', 'recruit', 'around', 'countri', '.', "'re", 'look', 'current', 'undergrad/grad', 'stem', 'student', "'re", 'stem', 'student', 'know', 'stem', 'student', ',', 'would', 'greatli', 'appreci',

##normalizing

###removing punctuation

In [ ]:
def remove_punctuation(data):
  new_data = []
  for words in data:
    new_words = [word for word in words if word.isalpha()]
    new_data.append(new_words)
  return new_data


In [ ]:
new_data = remove_punctuation(new_data)

In [ ]:
print(new_data)

[['said', 'felt', 'way', 'sugget', 'go', 'rest', 'trigger', 'ahead', 'youi', 'hypocondriac', 'like', 'decid', 'look', 'feel', 'doom', 'hope', 'mayb', 'get', 'suck', 'rabbit', 'hole', 'ludicr', 'conspiraci', 'stupid', 'psychic', 'test', 'new', 'age', 'someth', 'could', 'even', 'laugh', 'road', 'end', 'read', 'sens', 'doom', 'indic', 'variou', 'health', 'ailment', 'one', 'prone', 'top', 'doom', 'gloom', 'worri', 'heart', 'happen', 'physic', 'hour'], ['hey', 'sure', 'right', 'place', 'post', 'goe', 'current', 'student', 'intern', 'sandia', 'nation', 'lab', 'work', 'survey', 'help', 'improv', 'market', 'outreach', 'effort', 'mani', 'school', 'recruit', 'around', 'countri', 'look', 'current', 'stem', 'student', 'stem', 'student', 'know', 'stem', 'student', 'would', 'greatli', 'appreci', 'help', 'take', 'pass', 'along', 'short', 'survey', 'thank', 'everyon', 'help', 'take', 'survey', 'enter', 'draw', 'chanc', 'win', 'one', 'three', 'amazon', 'gc'], ['mom', 'hit', 'newspap', 'shock', 'would',

###all lowercase

In [ ]:
def lowercase(data):
  new_data = []
  for words in data:
    new_words = [w.lower() for w in words]
    new_data.append(new_words)
  return new_data

In [ ]:
new_data = lowercase(new_data)

In [ ]:
new_data2 = np.asarray(new_data)

<ipython-input-35-deb92250b6cb>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_data2 = np.asarray(new_data)


##label encoding

In [ ]:
def label_encode(data):
  new_data = []
  model = LabelEncoder()
  for words in data:
    new_words = model.fit_transform(words)
    new_data.append(new_words)
  return new_data

In [ ]:
new_data2 = label_encode(new_data)

In [ ]:
new_data = np.asarray(new_data2)

<ipython-input-38-3942dbd63e57>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_data = np.asarray(new_data2)


##one hot encoding

In [ ]:
def one_hot_encode(data):
  new_data = []
  for words in data:
    model = OneHotEncoder(sparse = False)
    words = words.reshape(len(words), 1)
    new_words = model.fit_transform(words)
    new_data.append(new_words)
  
  return new_data

In [ ]:
new_data3 = one_hot_encode(new_data)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in versi

#Preprocessing 

##Padding

In [16]:
data = get_data("data.pkl")

finding max length to know how much padding

In [ ]:
def get_max(array):
  max = array[0].size
  for a in array:
    if a.size > max:
      max = a.size
  return max

In [ ]:
max = get_max(data)

In [ ]:
print(max)

16215


In [ ]:
def pad_array(array, max):
  new_array = []
  for a in array:
    print(a.size)
    for i in range(a.size, max):
      a = np.append(a, 0)
    print(a.size)
    print(max) 
    new_array.append(a)
  return new_array

In [ ]:
pad = pad_array(new_data3, max)

Streaming output truncated to the last 5000 lines.
16215
16215
1804
16215
16215
2832
16215
16215
1845
16215
16215
2394
16215
16215
899
16215
16215
1584
16215
16215
594
16215
16215
1476
16215
16215
575
16215
16215
1428
16215
16215
999
16215
16215
1054
16215
16215
1054
16215
16215
2332
16215
16215
644
16215
16215
2160
16215
16215
169
16215
16215
928
16215
16215
957
16215
16215
1295
16215
16215
414
16215
16215
49
16215
16215
930
16215
16215
320
16215
16215
1184
16215
16215
2754
16215
16215
1584
16215
16215
1680
16215
16215
1360
16215
16215
323
16215
16215
1482
16215
16215
1330
16215
16215
2193
16215
16215
225
16215
16215
3286
16215
16215
870
16215
16215
1800
16215
16215
1716
16215
16215
1190
16215
16215
2679
16215
16215
1110
16215
16215
784
16215
16215
868
16215
16215
1824
16215
16215
900
16215
16215
952
16215
16215
2256
16215
16215
375
16215
16215
598
16215
16215
899
16215
16215
960
16215
16215
1368
16215
16215
1680
16215
16215
1760
16215
16215
306
16215
16215
1638
16215
16215
1026
16215

In [ ]:
def print_length(data, max):
  for a in data: 
    if a.size != max:
      print(a.size)

In [ ]:
print(data[438].size)

16215


In [ ]:
print(data.shape)

(2838,)


In [ ]:
print_length(data, max)

In [ ]:
for i1 in range(data.size - 1):
  for i2 in range(data[i1].size - 1):
    if np.isnan(data[i1][i2]) == True:
      print("i1: " + str(i1))
      print("i2: " + str(i2))

In [ ]:
print(data.shape)


(2838,)


In [ ]:
print(type(data))

<class 'numpy.ndarray'>


In [ ]:
def reconstruct(array): 
  new_array = []
  for a in array:
    new_array.append(a)
  new_array = np.asarray(new_array)
  return new_array

In [ ]:
new_data = reconstruct(data)

In [ ]:
print(new_data.shape)

(2838, 16215)


In [ ]:
save_data(new_data, "data.pkl")

##checking for strings

In [ ]:
dim1, dim2 = new_data.shape

In [ ]:
for i_array in range(dim1): 
  for i_val in range(dim2):
    val = new_data[i_array][i_val]
    if type(val) == type(""):
      print(val)
      print(i_array)
      print(i_val)

##pca

#Classification models

##Logistic Regression

In [17]:
def logistic_regression(x_train, y_train, x_test, y_test):
  model = LogisticRegression()
  model.fit(x_train, y_train)
  y_preds = model.predict(x_test)
  return model.score(x_test, y_test)

In [18]:
split = 2000
end = 2837

In [19]:
score = logistic_regression(data[:split], labels[:split], data[split:end], labels[split:end])

In [20]:
print(score)

0.4838709677419355


##LSTMs

##RNN

In [36]:
def rnn(x_train, y_train, x_test, y_test, num_epochs):
  dim1, dim2 = x_train.shape
  model = keras.Sequential()
  
  model.add(SimpleRNN(128, return_sequences = True, activation = "tanh", input_shape = (dim2, 1)))
  #latest addition: adding dropout layers
  #model.add(Dropout(0.2))
  model.add(SimpleRNN(16, activation = 'tanh', input_shape = (128,)))
  #model.add(Dropout(0.2))
  model.add(Dense(22, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5), activation = 'relu'))
  
  model.add(Dense(1, activation = 'sigmoid')) # change activation to sigmoid to keep values close to either 0 or 1 for binary classification
  model.add(Flatten())
  model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [keras.metrics.BinaryAccuracy(), keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives(), keras.metrics.TruePositives(), keras.metrics.TrueNegatives()]) # change loss function to BCE loss (binary crossentropy loss) to match with sigmoid
  #fitting model to training data and validating with test data
  model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = num_epochs)

In [ ]:
rnn(data[:split], labels[:split], data[split:end], labels[split:end], 30)

Epoch 1/30
 6/63 [=>............................] - ETA: 1:15:04 - loss: 0.7154 - binary_accuracy: 0.4792 - precision_6: 0.5040 - recall_6: 0.6238 - false_positives_6: 62.0000 - false_negatives_6: 38.0000 - true_positives_6: 63.0000 - true_negatives_6: 29.0000

##MLP


In [34]:
def cnn(x_train, y_train, x_test, y_test, num_epochs):
  dim1, dim2 = x_train.shape
  model = keras.Sequential()
  model.add(Dense(100, activation = 'relu', input_shape = (dim2, )))
  model.add(Dropout(0.3))
  model.add(Dense(20, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5), activation = 'relu'))
  model.add(Dropout(0.1))
  model.add(Dense(1, activation = 'sigmoid')) # change activation to sigmoid to keep values close to either 0 or 1 for binary classification
  model.add(Flatten())
  model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [keras.metrics.BinaryAccuracy(), keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives(), keras.metrics.TruePositives(), keras.metrics.TrueNegatives()]) # change loss function to BCE loss (binary crossentropy loss) to match with sigmoid
  #fitting model to training data and validating with test data
  model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = num_epochs)
  

In [35]:
cnn(data[:split], labels[:split], data[split:end], labels[split:end], 30)

Epoch 1/30
63/63 [==============================] - 6s 53ms/step - loss: 0.6989 - binary_accuracy: 0.5225 - precision_5: 0.5243 - recall_5: 0.9021 - false_positives_5: 853.0000 - false_negatives_5: 102.0000 - true_positives_5: 940.0000 - true_negatives_5: 105.0000 - val_loss: 0.6948 - val_binary_accuracy: 0.5329 - val_precision_5: 0.5456 - val_recall_5: 0.7258 - val_false_positives_5: 269.0000 - val_false_negatives_5: 122.0000 - val_true_positives_5: 323.0000 - val_true_negatives_5: 123.0000
Epoch 2/30
63/63 [==============================] - 3s 55ms/step - loss: 0.6052 - binary_accuracy: 0.7050 - precision_5: 0.6692 - recall_5: 0.8580 - false_positives_5: 442.0000 - false_negatives_5: 148.0000 - true_positives_5: 894.0000 - true_negatives_5: 516.0000 - val_loss: 0.7346 - val_binary_accuracy: 0.5066 - val_precision_5: 0.5342 - val_recall_5: 0.5618 - val_false_positives_5: 218.0000 - val_false_negatives_5: 195.0000 - val_true_positives_5: 250.0000 - val_true_negatives_5: 174.0000
Epoch 